# Hands On - Aprendizado Federado aplicado à Internet das Coisas

Nesse tutorial, vamos aprender como treinar uma rede neural para dados de Human Activity Recognition usando Flower e Pytorch.
No nosso exemplo, consideramos que o servidor e os dois clientes estão utilizando o mesmo modelo.

Para treinar o nosso exemplo de modelo federado, usaremos como contexto a classificação de atividades humanas (Human-Activity Recognition) com dados obtidos por meio de sensores presentes em dispositivos móveis. 
Nosso objetivo aqui é demonstrar os benefícios do uso de redes neurais para modelar a tarefa de HAR, com pouco ou nenhum conhecimento de domínio. 

## Dados utilizados na análise

Possuimos cerca de 20.000 leituras de sensores de 6 participantes realizando 5 ações diferentes. 
Cada leitura consiste em:

- medições de pose (roll, pitch, yaw), 
- acelerômetro (medições de aceleração linear), 
- giroscópio (medições da velocidade de rotação).

Cada feature é representada por um vetor 3D apontando na direção da leitura em um determinado passo de tempo para quatro sensores diferentes (cinto, braço, antebraço, haltere). 

Cada leitura foi normalizada em relação às amostras na mesma categoria no conjunto de dados e concatenadas em um único intervalo de tempo, formando assim um vetor de features de dimensão 40 x 1.

## Carregando os dados

Definimos aqui uma função auxiliar que carrega conjuntos de dados de treinamento e teste.

Parâmetros:
- data_root (str): Diretório que os datasets finais serão armazenados.

- train_batch_size (int): Tamanho definido para o mini-batch usado nos dados de treinamento.

- test_batch_size (int): Tamanho definido para o mini-batch usado nos dados de teste.

- cid (int): Client ID usado para selecionar uma partição específica.

- nb_clients (int): Número toral de clientes usados no treinamento.


Returns
- (train_loader, test_loader) (Tuple[DataLoader, DataLoader]): Tupla contendo DataLoaders para os conjuntos de treinamento e teste.

In [1]:
import pml

DATA_ROOT = "./data/pml-training.csv"
train_size = 64
test_size = 1000
c_id = 0
n_clients = 2 

train_loader, test_loader = pml.load_data(
                                data_root = DATA_ROOT,
                                train_batch_size = train_size,
                                test_batch_size = test_size,
                                cid = c_id,
                                nb_clients = n_clients + 1
)

## O modelo

O vetor de características é então passado por um CNN 1D com 3 camadas conv e 2 camadas totalmente conectadas. Cada camada é seguida por uma função de ativação ReLU e uma camada de dropout. 
Nossa taxa de aprendizado inicial é 0,003, com uma taxa de decaimento de 0,95 por época. 
Usamos uma taxa de divisão de 0,2 para dados de treinamento e validação (80% de treinamento, 20% de validação) e alcançamos 75% de precisão após 300 épocas.

In [3]:
import torch.nn as nn

class HARmodel(nn.Module):
    """Model for human-activity-recognition."""
    def __init__(self, input_size, num_classes):
        super().__init__()

        # Extract features, 1D conv layers
        self.features = nn.Sequential(
            nn.Conv1d(input_size, 64, 1),
            nn.ReLU(),
            nn.Dropout(),
            nn.Conv1d(64, 64, 1),
            nn.ReLU(),
            nn.Dropout(),
            nn.Conv1d(64, 64, 1),
            nn.ReLU(),
            nn.Flatten(),
            )
        # Classify output, fully connected layers
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, num_classes),
            )

    def forward(self, x):
        x = self.features(x)
        out = self.classifier(x)
        return out

A rede consiste em 3 camadas convolucionais e 2 camadas totalmente conectadas, uma arquitetura relativamente superficial para os padrões atuais. 

## Cliente Flower

O primeiro passo é definir a aloção de dispositivos no pyTorch: 

In [4]:
import torch

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Quando o servidor seleciona um dispositivo específico do ambiente federado para realizar um treinamento, ele envia as instruções pela rede, por meio de uma interface chamada Client.
Assim,o cliente recebe as instruções do servidor e chama um dos métodos desta classe para executar seu código (ou seja, para treinar a sua rede neural local).

O framework Flower fornece uma classe chamada NumPyClient, que torna mais fácil implementar a interface do cliente quando utilizamos PyTorch. Implementar NumPyClient geralmente significa definir os seguintes métodos:

- **get_parameters**: retorna o peso do modelo como uma lista de ndarrays 

- **set_parameters** (opcional): atualiza os pesos do modelo local com os parâmetros recebidos do servidor 

- **fit**: define os pesos do modelo local, treina o modelo localmente e receber o update dos pesos locais
    
    Parameters: Parameters sent by the server to be used during training.
    
    Returns:Set of variables containing the new set of weights and information the client.

- **evaluate**: testa o modelo local.

    Parameters: Parameters sent by the server to be used during testing.
    
    Returns: Information the clients testing results.

In [5]:
import flwr as fl
import torch
from torchvision import datasets, transforms

class HARClient(fl.client.Client):

    def __init__(
        self,
        cid: int,
        train_loader: datasets,
        test_loader: datasets,
        epochs: int,
        device: torch.device = torch.device("cpu"),
    ) -> None:
        self.model = HARmodel(40, 5).to(device)
        self.cid = cid
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.device = device
        self.epochs = epochs

    def get_weights(self) -> fl.common.Weights:
        return [val.cpu().numpy() for _, val in self.model.state_dict().items()]

    def set_weights(self, weights: fl.common.Weights) -> None:
        state_dict = OrderedDict(
            {
                k: torch.Tensor(v)
                for k, v in zip(self.model.state_dict().keys(), weights)
            }
        )
        self.model.load_state_dict(state_dict, strict=True)

    def get_parameters(self) -> fl.common.ParametersRes:
        weights: fl.common.Weights = self.get_weights()
        parameters = fl.common.weights_to_parameters(weights)
        return fl.common.ParametersRes(parameters=parameters)

    def fit(self, ins: fl.common.FitIns) -> fl.common.FitRes:

        # Set the seed so we are sure to generate the same global batches
        # indices across all clients
        np.random.seed(123)

        weights: fl.common.Weights = fl.common.parameters_to_weights(ins.parameters)
        fit_begin = timeit.default_timer()

        # Set model parameters/weights
        self.set_weights(weights)

        # Train model
        num_examples_train: int = train(
            self.model, self.train_loader, epochs=self.epochs, device=self.device, cid=self.cid
        )

        # Return the refined weights and the number of examples used for training
        weights_prime: fl.common.Weights = self.get_weights()
        params_prime = fl.common.weights_to_parameters(weights_prime)
        fit_duration = timeit.default_timer() - fit_begin
        return fl.common.FitRes(
            parameters=params_prime,
            num_examples=num_examples_train,
            num_examples_ceil=num_examples_train,
            fit_duration=fit_duration,
        )

    def evaluate(self, ins: fl.common.EvaluateIns) -> fl.common.EvaluateRes:
        weights = fl.common.parameters_to_weights(ins.parameters)

        # Use provided weights to update the local model
        self.set_weights(weights)
        (
            num_examples_test,
            test_loss,
            accuracy,
        ) = test(self.model, self.test_loader, device=self.device)
        print(
            f"Client {self.cid} - Evaluate on {num_examples_test} samples: Average loss: {test_loss:.4f}, Accuracy: {100*accuracy:.2f}%\n"
        )

        # Return the number of evaluation examples and the evaluation result (loss)
        return fl.common.EvaluateRes(
            num_examples=num_examples_test,
            loss=float(test_loss),
            accuracy=float(accuracy),
        )

## Server Flower

In [10]:
def eval(w):
    train_loader, test_loader = pml.load_data(
        data_root = DATA_ROOT,
        train_batch_size = 64,
        test_batch_size = 4,
        cid = 5,
        nb_clients = 6,
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    server = HARClient(
        cid = 999,
        train_loader = train_loader,
        test_loader = test_loader,
        epochs = 1,
        device = device
    )
    server.set_weights(w)
    return test(server.model, train_loader, device)

In [11]:
strategy = fl.server.strategy.FedAvg(
        eval_fn = eval,
)

fl.server.start_server(config={"num_rounds": 10}, strategy = strategy)

INFO flower 2021-06-19 08:57:57,577 | app.py:73 | Flower server running (insecure, 10 rounds)
INFO flower 2021-06-19 08:57:57,579 | server.py:118 | Getting initial parameters


KeyboardInterrupt: 

## Run Clients

In [12]:
# Instantiate and start client

epochs = 14
server_address = "[::]:8080"

client = HARClient(
        cid = c_id,
        train_loader = train_loader,
        test_loader = test_loader,
        epochs = epochs,
        device = DEVICE,
)

fl.client.start_client(server_address, client)

DEBUG flower 2021-06-19 08:59:27,184 | connection.py:36 | ChannelConnectivity.IDLE
DEBUG flower 2021-06-19 08:59:27,186 | connection.py:36 | ChannelConnectivity.CONNECTING
INFO flower 2021-06-19 08:59:27,188 | app.py:61 | Opened (insecure) gRPC connection
DEBUG flower 2021-06-19 08:59:27,188 | connection.py:36 | ChannelConnectivity.READY
DEBUG flower 2021-06-19 09:05:20,748 | connection.py:68 | Insecure gRPC channel closed


KeyboardInterrupt: 